In [27]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert", device=0) # device=0 to use GPU

# result = pipe("I am bullish on $AAPL", return_all_scores=True)


In [28]:
# Test case
result = pipe("I am bullish on $AAPL", return_all_scores=True)


c:\Users\chand\miniconda3\envs\resurgence\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [29]:
import pandas as pd

# Replace 'path_to_first_csv.csv' and 'path_to_second_csv.csv' with the actual file paths
file_path1 = '2022_2023_data.csv'
file_path2 = '2024_data.csv'

# Read the CSV files
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# Concatenate the DataFrames
merged_df = pd.concat([df1, df2], axis=0)

# Optionally reset the index if you want a clean sequential index in the new DataFrame
merged_df.reset_index(drop=True, inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('2024_merged_data.csv', index=False)


In [30]:
import polars as pl

df = pl.read_csv('2024_merged_data.csv')
print(df.head(1))


shape: (1, 14)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ title     ┆ url       ┆ time_publ ┆ authors   ┆ … ┆ overall_s ┆ overall_s ┆ ticker_se ┆ date     │
│ ---       ┆ ---       ┆ ished     ┆ ---       ┆   ┆ entiment_ ┆ entiment_ ┆ ntiment   ┆ ---      │
│ str       ┆ str       ┆ ---       ┆ str       ┆   ┆ score     ┆ label     ┆ ---       ┆ i64      │
│           ┆           ┆ str       ┆           ┆   ┆ ---       ┆ ---       ┆ str       ┆          │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ str       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Apple,    ┆ https://w ┆ 20221231T ┆ ['Michael ┆ … ┆ -0.079733 ┆ Neutral   ┆ [{'ticker ┆ 20221231 │
│ AMC,      ┆ ww.benzin ┆ 190310    ┆ Cohen']   ┆   ┆           ┆           ┆ ':        ┆          │
│ Meta, And ┆ ga.com/ne ┆           ┆           ┆   ┆           ┆           

In [31]:
from datasets import Dataset

# summary_list = df['summary'].to_list()
# summary_list
data_dict = {col: df[col].to_list() for col in df.columns}

dataset = Dataset.from_dict(data_dict)
len(dataset)


20752

In [32]:
# result = pipe(summary_list, return_all_scores=True)
from transformers.pipelines.pt_utils import KeyDataset

outputs = []
for out in pipe(KeyDataset(dataset, "summary"), batch_size=64, truncation="only_first"):
    outputs.append([out])
    # print(out)
    break


In [33]:
df = df.with_columns(pl.Series("Finbert_vals", outputs))


In [34]:
file_path = '2024_finbert.json'
json_str = df.write_json()

# Write the JSON string to a file
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(json_str)



In [35]:
import pandas as pd
curr_df = pd.read_json('2024_finbert.json')

curr_df = df.to_pandas()
curr_df.to_csv('2024_data_finbert.csv', index = False)